In [24]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import pandas as pd

# 1. Directory definition

In [17]:
output_directory = "out" # output data will be saved here, need to be created beforehand
os.chdir('/home/klegoff/Documents/test_data') # raw data location

In [2]:
#get data if needed
#!wget https://common-voice-data-download.s3.amazonaws.com/cv_corpus_v1.tar.gz

In [19]:
!ls #wav files are in clips directory

clips	 invalidated.tsv  reported.tsv	train.tsv
dev.tsv  other.tsv	  test.tsv	validated.tsv


In [44]:
datapath = "clips"
file_list = os.listdir(datapath)  # list of all files to be processed

# 2. Processing

In [39]:
N_mfcc = 15 #number of mfcc to compute, usually 10-15 coefficients, according to 

In [128]:
def cut_data(data,samplerate,n_ms=0.05):
    """
    cut the data in 50 ms bins (by default)
    """
    step = int(n_ms*samplerate) # we take 50 ms time interval

    # cut the first element
    i=0
    cut =data[i*step : (i+1)*step]
    cut_data = [cut]
    
    while True:
        i+=1
        cut = data[i*step : (i+1)*step]
        if cut.shape[0] == 0 :
            break
        cut_data.append(cut)
                
    return cut_data

In [133]:
def save(f,d):
    """
    save the data in the file
    f:filename
    d:data
    """
    with open(f,'wb'):
        np.save(f,d)


def compute_mfcc(file,n_mfcc=15):
    """
    compute mfcc array
    to be multiprocessed ? probably needed for big volume
    """
    #load mp3/wav file
    data, samplerate = librosa.load(datapath +os.sep+file)
    
    #cut data in 50ms intervals
    sliced_data = cut_data(data, samplerate)
    
    for i in range(len(sliced_data)):
        red_data = sliced_data[i]
        
        #compute mfcc
        mfcc = librosa.feature.mfcc(red_data,n_mfcc=n_mfcc)

        # save in the output directory
        save(output_directory+os.sep+file[:-4]+"_"+str(i)+".npy", mfcc)

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_worker=4) as executor:
    executor.map(compute_mfcc, file_list)

In [106]:
# do we need to filter out some uninformative sequences ?

(2000,)